In [9]:
### basic libraries
import os
from json import dumps

### carto libraries
import shapefile
import geopandas

from app.scripts.topojson import topojson

from pyproj import Proj, transform
inProj  = Proj(init='epsg:2154') # proj in  : Lambert 93
outProj = Proj(init='epsg:4326') # proj out : WSG 84

### TO DO : find topojson libraries to reduce output file size


ImportError: No module named app_vars

In [2]:
### basic folders addresses and names
cwd = os.getcwd()

data_folder = "app/static/data"
carto_folder = "carto"

carto_path = os.path.join(cwd, data_folder, carto_folder)

print "-- cwd :", cwd
print "-- carto path : ", carto_path


-- cwd : /Users/jpy/Dropbox/_FLASK/concours_pesticides
-- carto path :  /Users/jpy/Dropbox/_FLASK/concours_pesticides/app/static/data/carto


In [3]:
# cf : https://pypi.python.org/pypi/pyshp
# cf : http://gis.stackexchange.com/questions/183795/how-do-i-select-shapefiles-to-be-converted-to-geojson-in-folder-with-multiple-sh
# cf : https://github.com/mlaloux/Python-geo_interface-applications/blob/master/PyShp_geointerface.py

# cf : https://glenbambrick.com/tag/pyshp/
# cf : https://glenbambrick.com/2016/01/24/reproject-shapefile/

In [4]:
### SHP files 
_shp  = ".shp"
_json = ".json"
_copy = "_copy"

water_shp_fname = "PolygMasseDEauSouterraine"

## TO DO : add departements .shp 

In [5]:
### reading/writing - converting shp files to geojson
    
def geofile_path(filename, extension):
    path = os.path.join(carto_path , filename + extension )
    print path
    return path


In [6]:
'''
def readSHP(filename):  
    # generator 
    reader = shapefile.Reader( root_carto_folder+ filename + _shp )  
    fields = reader.fields[1:]  
    field_names = [field[0] for field in fields]  
    for sr in reader.shapeRecords():  
        geom = sr.shape.__geo_interface__  
        atr = dict(zip(field_names, sr.record))  
        yield dict(geometry=geom,properties=atr)    
'''

def readSHP(filename) :
    
    # read the shapefile
    reader = shapefile.Reader( geofile_path( water_shp_fname, _shp ) )
    fields = reader.fields[1:]
    field_names = [field[0] for field in fields]

    buffer = []
    for sr in reader.shapeRecords():
       atr = dict(zip(field_names, sr.record))
       geom = sr.shape.__geo_interface__
       buffer.append(dict(type="Feature", geometry=geom, properties=atr)) 

    # write the GeoJSON file (copy)
    geojson = open( os.path.join( carto_path, filename + _copy + _json), "w")
    geojson.write(dumps({"type": "FeatureCollection", "features": buffer}, indent=2) + "\n")
    geojson.close()
    

In [7]:
gdf = geopandas.read_file( geofile_path( water_shp_fname, _shp ))

gdf.crs

{u'ellps': u'GRS80',
 u'lat_0': 46.5,
 u'lat_1': 49,
 u'lat_2': 44,
 u'lon_0': 3,
 u'no_defs': True,
 u'proj': u'lcc',
 u'units': u'm',
 u'x_0': 700000,
 u'y_0': 6600000}

In [8]:
#gdf.plot()

In [18]:
### test read/write /// .shp --> .json

#readSHP(water_shp_fname) ### problem : 1.16 Go file !!!